In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [34]:
import pandas as pd

# File paths
FUNDING_FILE = './data/funding_updates_final.csv'
BORROWING_FILE = './data/borrowing_updates_final.csv'
OPEN_INTEREST_FILE = './data/open_interest_updates_final.csv'

# ETH/USD market address
target_market = '0x70d95587d40a2caf56bd97485ab3eec10bee6336'

print(f"Loading data for market: {target_market}")

# Load the data
funding_df = pd.read_csv(FUNDING_FILE)
borrowing_df = pd.read_csv(BORROWING_FILE)
open_interest_df = pd.read_csv(OPEN_INTEREST_FILE)

print(f"Loaded {len(funding_df)} funding, {len(borrowing_df)} borrowing, {len(open_interest_df)} open interest records")

# Filter everything by our target market
market_funding = funding_df[funding_df['market'].str.lower() == target_market.lower()].copy()
market_borrowing = borrowing_df[borrowing_df['market'].str.lower() == target_market.lower()].copy()
market_open_interest = open_interest_df[open_interest_df['market'].str.lower() == target_market.lower()].copy()

print(f"After filtering: {len(market_funding)} funding, {len(market_borrowing)} borrowing, {len(market_open_interest)} OI records")

# Save to new files
market_funding.to_csv('./data/filtered_by_market_funding_updates.csv', index=False)
market_borrowing.to_csv('./data/filtered_by_market_borrowing_updates.csv', index=False)
market_open_interest.to_csv('./data/filtered_by_market_open_interest_updates.csv', index=False)

# Quick data check
print(f"\nFunding isLong: {market_funding['isLong'].value_counts().to_dict()}")
print(f"Borrowing isLong: {market_borrowing['isLong'].value_counts().to_dict()}")
print(f"Open Interest isLong: {market_open_interest['isLong'].value_counts().to_dict()}")


Loading data for market: 0x70d95587d40a2caf56bd97485ab3eec10bee6336
Loaded 704325 funding, 882792 borrowing, 343118 open interest records
After filtering: 105918 funding, 111900 borrowing, 49795 OI records

Funding isLong: {True: 82778, False: 23140}
Borrowing isLong: {True: 55950, False: 55950}
Open Interest isLong: {True: 30901, False: 18894}


In [35]:
import pandas as pd

# File paths
FUNDING_FILE = './data/funding_updates_final.csv'
BORROWING_FILE = './data/borrowing_updates_final.csv'
OPEN_INTEREST_FILE = './data/open_interest_updates_final.csv'

# ETH/USD market address
target_market = '0x70d95587d40a2caf56bd97485ab3eec10bee6336'

print(f"Loading data for market: {target_market}")

# Load the data
funding_df = pd.read_csv(FUNDING_FILE)
borrowing_df = pd.read_csv(BORROWING_FILE)
open_interest_df = pd.read_csv(OPEN_INTEREST_FILE)

print(f"Loaded {len(funding_df)} funding, {len(borrowing_df)} borrowing, {len(open_interest_df)} open interest records")

# Filter everything by our target market
market_funding = funding_df[funding_df['market'].str.lower() == target_market.lower()].copy()
market_borrowing = borrowing_df[borrowing_df['market'].str.lower() == target_market.lower()].copy()
market_open_interest = open_interest_df[open_interest_df['market'].str.lower() == target_market.lower()].copy()

print(f"After filtering: {len(market_funding)} funding, {len(market_borrowing)} borrowing, {len(market_open_interest)} OI records")

# Save to new files
market_funding.to_csv('./data/filtered_by_market_funding_updates.csv', index=False)
market_borrowing.to_csv('./data/filtered_by_market_borrowing_updates.csv', index=False)
market_open_interest.to_csv('./data/filtered_by_market_open_interest_updates.csv', index=False)

# Quick data check
print(f"\nFunding isLong: {market_funding['isLong'].value_counts().to_dict()}")
print(f"Borrowing isLong: {market_borrowing['isLong'].value_counts().to_dict()}")
print(f"Open Interest isLong: {market_open_interest['isLong'].value_counts().to_dict()}")


Loading data for market: 0x70d95587d40a2caf56bd97485ab3eec10bee6336
Loaded 704325 funding, 882792 borrowing, 343118 open interest records
After filtering: 105918 funding, 111900 borrowing, 49795 OI records

Funding isLong: {True: 82778, False: 23140}
Borrowing isLong: {True: 55950, False: 55950}
Open Interest isLong: {True: 30901, False: 18894}


In [40]:
from web3 import Web3
import pandas as pd
from tqdm import tqdm
import time

RPC_ENDPOINT = 'https://ultra-special-sound.arbitrum-mainnet.quiknode.pro/471f8faf6838742a15e50d7f9161adc72f8738c9/'
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))

def get_timestamp_by_block(block_number: int):
    """Get timestamp for a given block number with error handling"""
    try:
        return w3.eth.get_block(block_number).timestamp
    except Exception as e:
        print(f"Error getting timestamp for block {block_number}: {e}")
        return None

# Load the filtered data first
print("Loading filtered data...")
market_funding = pd.read_csv('./data/filtered_by_market_funding_updates.csv')
market_borrowing = pd.read_csv('./data/filtered_by_market_borrowing_updates.csv')
market_open_interest = pd.read_csv('./data/filtered_by_market_open_interest_updates.csv')

print(f"Loaded {len(market_funding)} funding, {len(market_borrowing)} borrowing, {len(market_open_interest)} open interest records")

# Test Web3 connection
print("Testing Web3 connection...")
try:
    latest_block = w3.eth.block_number
    print(f"Connected! Latest block: {latest_block}")
except Exception as e:
    print(f"Connection failed: {e}")
    exit()

# Get ALL unique blocks from all datasets (avoid duplicate fetching)
print("Finding all unique blocks...")
all_unique_blocks = set()
all_unique_blocks.update(market_funding['blockNumber'].unique())
all_unique_blocks.update(market_borrowing['blockNumber'].unique())
all_unique_blocks.update(market_open_interest['blockNumber'].unique())

print(f"Total unique blocks to fetch: {len(all_unique_blocks)}")

# Fetch timestamps for all unique blocks
print("Fetching timestamps...")
block_to_timestamp = {}

for block_num in tqdm(all_unique_blocks, desc="Fetching timestamps"):
    timestamp = get_timestamp_by_block(int(block_num))
    if timestamp is not None:
        block_to_timestamp[block_num] = timestamp
    
    time.sleep(0.1)  # Reduced delay since we're being more efficient

print(f"Successfully fetched {len(block_to_timestamp)} timestamps")

# Add timestamps to all datasets using the same mapping
print("Adding timestamps to datasets...")
market_funding['timestamp'] = market_funding['blockNumber'].map(block_to_timestamp)
market_funding['datetime'] = pd.to_datetime(market_funding['timestamp'], unit='s')

market_borrowing['timestamp'] = market_borrowing['blockNumber'].map(block_to_timestamp)
market_borrowing['datetime'] = pd.to_datetime(market_borrowing['timestamp'], unit='s')

market_open_interest['timestamp'] = market_open_interest['blockNumber'].map(block_to_timestamp)
market_open_interest['datetime'] = pd.to_datetime(market_open_interest['timestamp'], unit='s')

# Save all three files
market_funding.to_csv('./data/market_funding_with_timestamps.csv', index=False)
market_borrowing.to_csv('./data/market_borrowing_with_timestamps.csv', index=False)
market_open_interest.to_csv('./data/market_open_interest_with_timestamps.csv', index=False)

print("\nCompleted! Files saved:")
print(f"Funding: {market_funding.shape}")
print(f"Borrowing: {market_borrowing.shape}")
print(f"Open Interest: {market_open_interest.shape}")

# Check for any missing timestamps
missing_funding = market_funding['timestamp'].isna().sum()
missing_borrowing = market_borrowing['timestamp'].isna().sum()
missing_oi = market_open_interest['timestamp'].isna().sum()

if missing_funding + missing_borrowing + missing_oi > 0:
    print(f"\nWarning: Missing timestamps - Funding: {missing_funding}, Borrowing: {missing_borrowing}, OI: {missing_oi}")

# Show date ranges
if len(market_funding) > 0:
    print(f"\nDate range: {market_funding['datetime'].min()} to {market_funding['datetime'].max()}")


Loading filtered data...
Loaded 105918 funding, 111900 borrowing, 49795 open interest records
Testing Web3 connection...
Connected! Latest block: 350511842
Finding all unique blocks...
Total unique blocks to fetch: 54368
Fetching timestamps...


Fetching timestamps:  92%|█████████▏| 49757/54368 [2:40:36<1:10:06,  1.10it/s]

Error getting timestamp for block 323604715: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Fetching timestamps: 100%|██████████| 54368/54368 [2:53:57<00:00,  5.21it/s]  


Successfully fetched 54367 timestamps
Adding timestamps to datasets...

Completed! Files saved:
Funding: (105918, 8)
Borrowing: (111900, 7)
Open Interest: (49795, 8)


Date range: 2025-03-30 06:58:48 to 2025-05-29 08:30:46
